# Wav2Vec2でキーワードスポッティング(?)する

キーワードスポッティングの要件

1. キーワードは事前に登録される（複数）
1. 多少の認識エラーを許容し、できれば、認識の正確さを数値化したい

In [ ]:
from transformers import AutoTokenizer, AutoFeatureExtractor,Wav2Vec2ForCTC,Wav2Vec2Processor
import torch

MODEL_NAME = "AndrewMcDowell/wav2vec2-xls-r-300m-japanese"
#MODEL_NAME = "facebook/wav2vec2-lv-60-espeak-cv-ft"
model = Wav2Vec2ForCTC.from_pretrained(MODEL_NAME,torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
feature_extractor = AutoFeatureExtractor.from_pretrained(MODEL_NAME)
processor=Wav2Vec2Processor.from_pretrained(MODEL_NAME)

In [ ]:
import soundfile as sf
import librosa

def readAudioFile(audio_path):
    audio, sample_rate = sf.read(audio_path)
    if audio.ndim > 1:
        audio = audio[:, 0]
    if sample_rate != 16000:
        audio = librosa.resample(audio, orig_sr=sample_rate, target_sr=16000)
    return audio.astype("float16")


In [ ]:
model.to("cuda:0")

# 001-10.wavで試して時間を計測する
import time
start_time = time.time()
# LOANWORD128_060:フランスの作曲家ギィ・ロパルツが作曲した。
# LOANWORD128_061:ギェンツェン・ノルブは、中華人民共和国、チベット自治区出身の、チベット族の僧である。
# LOANWORD128_062:どれがクァディ族のものか定かではない。
# LOANWORD128_063:クィーンアリア国際空港は、ヨルダンの首都アンマンにある国際空港である。
# LOANWORD128_064:クェゼリン島の戦いとは，日本軍の守るクェゼリン環礁へ、アメリカ軍が侵攻して行われた戦闘である。
# LOANWORD128_065:クォン・サンウは、韓国の俳優である。
# LOANWORD128_066:グァルネリは、イタリア出身の弦楽器製作者一族、または、彼らが制作した弦楽器である。
# LOANWORD128_067:グィネヴィアは小惑星帯の外縁部付近に位置する小惑星である。
# LOANWORD128_068:親友となったザフトの英雄グゥド・ヴェイアとの戦闘で損傷した。
# LOANWORD128_069:グィネヴィアは、伝説的な人物で、アーサー王の王妃。
# LOANWORD128_070:グォ・ヨウは，中国北京出身の男性俳優。
wavs=range(60,71)
for i in wavs:
    audio = readAudioFile(f"/mnt/e/dataset/jsut_ver1.1/loanword128/wav/LOANWORD128_{i:03d}.wav")
    input_values = feature_extractor(audio, sampling_rate=16000,return_tensors="pt").input_values.to("cuda:0")
    logits = model(input_values)[0]
    pred_ids = torch.argmax(logits, axis=-1)
    text=processor.batch_decode(pred_ids)
    print(f"{i:04d}: {text[0]}")

elapsed_time = time.time() - start_time
avg_time = elapsed_time / len(wavs)
print(f"elapsed_time:{elapsed_time}[sec] avg_time:{avg_time}[sec/wav]")

In [ ]:
# LOANWORD128_064:クェゼリン島の戦いとは，日本軍の守るクェゼリン環礁へ、アメリカ軍が侵攻して行われた戦闘である。
# 0064: ケェヴェリンとうのたたかいとは、にっほんぐんのまもるクェヴェリンかんしょうへアメリカぐんがしんこうしたおかなわれたせんとうである。
# キーワード登録によって「クェゼリン」を検出してみる

# シーケンスに含まれるID、全てにbiasを加えてデコードしてみる
audio = readAudioFile(f"/mnt/e/dataset/jsut_ver1.1/loanword128/wav/LOANWORD128_064.wav")
input_values = feature_extractor(audio, sampling_rate=16000,return_tensors="pt").input_values.to("cuda:0")
logits = model(input_values)[0]
pred_ids = torch.argmax(logits, axis=-1)
org_text=processor.batch_decode(pred_ids)

In [ ]:
# logitsを受け取り、wordのbiasを加えたlogitsを返す関数
def simple_bias_strategy(word,logits,bias):
    # idに変換
    ids=tokenizer.convert_tokens_to_ids(tokenizer.tokenize(word))
    logits[0,:,ids]+=bias
    return logits

logits = model(input_values)[0]
logits = simple_bias_strategy("クェゼリン",logits,5.0)
pred_ids_with_bias = torch.argmax(logits, axis=-1)

#decode結果を比較
[org_text,processor.batch_decode(pred_ids_with_bias)]


正解：クェゼリン　に対して

ケェヴェリン　→　クェェリン
クェヴェリン　→　クェゼェリン

になった。惜しい感じ

ヴェ→ゼ
になれば正解する雰囲気がある。

もっと詳細に結果を検討する

In [ ]:
# argmaxでデコードされた文字列を探し、その部分の上位N件のトークンを表示する
def findTopKByDecodedText(needle,logits,k=5):
    needle_ids=tokenizer.convert_tokens_to_ids(tokenizer.tokenize(needle))
    argmax_result = torch.argmax(logits, axis=-1).cpu()
    argmax_result = argmax_result[0]
    #argmax_resultからneedle_idsが全てマッチするかをチェック（PADであった場合無視する）、マッチしていれば先頭のindexを返す
    start_index=-1
    end_index=-1
    needle_index=0
    for i in range(len(argmax_result)-len(needle_ids)):
        end_index = i       
        current_id = argmax_result[i]
        if current_id==tokenizer.pad_token_id:
            continue
        if current_id==needle_ids[needle_index]:
            #print(f"match {i} {current_id} {needle[needle_index]}")
            if needle_index==0:
                start_index=i
            needle_index+=1            
            if needle_index==len(needle_ids):
                break
        elif current_id == last_id:
            #print(f"dup {i} {current_id} {needle[needle_index]}")            
            continue
        else:
            if needle_index>0:
                #print(f"unmatch {i} {current_id} {needle[needle_index]}")
                pass
            start_index=-1
            needle_index=0
        last_id=current_id

    if start_index==-1:
        return None            
    #start_indexからend_index+1のk件のトークンを取得
    print(f"start_index={start_index} end_index={end_index}")
    topk_result = torch.topk(logits[0,start_index:end_index+1,:],k)

    result_token=[]
    result_score=[]
    #トークンを文字列に変換
    for i,ids in enumerate(topk_result.indices):
        result_token.append(tokenizer.convert_ids_to_tokens(ids.tolist()))
        result_score.append(topk_result.values[i].tolist())
    return [result_token,result_score]

logits = model(input_values)[0]
[
    findTopKByDecodedText("ケェヴェリン",logits,k=5),
    findTopKByDecodedText("クェェリン", simple_bias_strategy("クェゼリン",logits,5.0),k=5)
]


    

In [ ]:
# 異なる戦略を試す。

# 単純にシーケンス内の全部のIDにbiasを加えても認識結果は良くならないので、順序を考慮したbiasを加える
# 1.全体にシーケンス内の最初のIDにbiasを加える
# 2.最初のIDがtopに来たら、次のIDにbiasを加える
# 3.最後のIDまで一致するまでbiasを足す
# 4.logitsの最後までやる
# この処理を行ったlogitsをdecodeする

def sequence_bias_strategy(word,logits,bias):
    logits = logits.clone()
    ids=tokenizer.convert_tokens_to_ids(tokenizer.tokenize(word))
    #最初のidをbiasで追加する
    logits[0,:,ids[0]]+=bias
    id_index=0
    time_step_length=len(logits[0])-1
    for i in range(time_step_length):
        max_id = torch.argmax(logits[0,i,:])
        if max_id == ids[id_index]:
            # 一致したら次のidに進む
            id_index+=1
            if id_index == len(ids):
                # 最後のidまで一致したので最初からやり直す
                id_index=0
                continue
        else:
            #　一致しなかった場合
            # 違う認識結果の場合と、前回の繰り返しと、PADの場合がある。
            if max_id == tokenizer.pad_token_id:
                #PADの場合は何もしない
                pass
            elif id_index > 0 and max_id == ids[id_index-1]:
                # 前回と同じ場合はPADとidsにbiasを加える
                logits[0,i,tokenizer.pad_token_id]+=bias
                logits[0,i,ids[id_index]]+=bias
                pass
            else:
                # 違う認識結果の場合なのか、認識しそこなっているかの判断はできない…
                logits[0,i,ids[id_index]]+=bias
                pass
    return logits

logits = model(input_values)[0]
# logitsのコピー
processor.batch_decode(torch.argmax(sequence_bias_strategy("クェゼリン",logits,5.0), axis=-1))

シーケンスバイアスは一定の効果はあるようだが、bias設定ステージでは、違う言葉なのか、認識をミスっているのかはわからないので、どうにもならない気がする。

decodeで一意に結果を求めるのではなく、キーワードのシーケンスの累積スコア（または対数確率の合算）で、キーワードが出現した可能性を数値したほうが合理的かもしれない。

ひとまず、シーケンス[A,B,C]についてスコアの累積を行うことを考えてみると、

1. キーワードが含まれているかもしれない、logits内のtime_stepの範囲を決める（どうやって？という話だが、総当たりでも、topKで最初と最後のIDを見つけても良い
2. 範囲内で、Aのスコアを得る。
3. 次のtime_stepでBのスコアを得るが、Aの繰り返し、またはPADよりも低かった場合は、次のステップへ行く
4. BのスコアがAまたはPADよりも高い場合は、Bのスコアを得る
5. Cのスコアに対しても同じ処理をする
6. マッチが終わるか、logits範囲が終わったらスコアを返す（残りは無視

このスコアの合算値は妥当だろうか？（logits内の別time_stepのスコアが合算できるものかはさておいて…）

例えばキーワードが
「ボボボーボ」
であれば、3の処理があるため、
ボーボの合算値しか出ないことがある。まあ、連続する文字の認識についてはスコアにゲタを吐かせてもいいのかしれないが…

ひとまず、今は考えない。

In [ ]:
logits = model(input_values)[0]
torch.softmax(logits[0,:,:],dim=1)
topk_result = torch.topk(logits[0,:,:],10)
#topk_result.indicesからPADトークンを探す
result=torch.where(topk_result.indices==tokenizer.pad_token_id)
18 in result[0]

In [ ]:
test=torch.tensor([[0,1,2,3,4,5,6,7,8,9],[10,9,8,7,6,5,4,3,2,1]])
# 8のindexを取得
result=torch.where(test==8)
0 in result[0]

In [ ]:
# 単純なスコア
def get_score_simple(logits,index,id):
    return logits[index,id]

# top10の何位かでスコアを決める
def get_score_by_rank(logits,index,id):
    top10 = torch.topk(logits[index,:],10)
    rank=torch.where(top10.indices == id)
    if len(rank[0])==0:
        return 0
    return 10 - rank[0]

# probの合計でスコアを決める
def get_score_by_prob(logits,index,id):
    return torch.softmax(logits[index,:],dim=0)[id]
    

def calc_score(word,sub_logits,get_score):
    ids=tokenizer.convert_tokens_to_ids(tokenizer.tokenize(word))
    score=0
    id_index=0
    for i in range(len(sub_logits)):
        target_score=get_score(sub_logits,i,ids[id_index])
        if id_index>0:
            last_id_score=get_score(sub_logits,i,ids[id_index-1])
        else:
            last_id_score=0
        pad_score=get_score(sub_logits,i,tokenizer.pad_token_id)
        if target_score>last_id_score and target_score>pad_score:
            score+=target_score
            id_index+=1
            if id_index==len(ids):
                break
    return score

logits = model(input_values)[0]

[
[
    calc_score("クェゼリン",logits[0,18:37,:],get_score_simple),
    calc_score("ケェヴェリン",logits[0,18:37,:],get_score_simple),
    calc_score("テスト",logits[0,18:37,:],get_score_simple)
],
[
    calc_score("クェゼリン",logits[0,18:37,:],get_score_by_rank),
    calc_score("ケェヴェリン",logits[0,18:37,:],get_score_by_rank),
    calc_score("テスト",logits[0,18:37,:],get_score_by_rank)
],
[
    calc_score("クェゼリン",logits[0,18:37,:],get_score_by_prob),
    calc_score("ケェヴェリン",logits[0,18:37,:],get_score_by_prob),
    calc_score("テスト",logits[0,18:37,:],get_score_by_prob)
]
]

In [ ]:
# 普通にscoreの合算で差異は出るので、後は、キーワード毎の比較ができるように正規化すれば良い
# 今のやり方だと長いシーケンスはそれだけスコアが高くなるので、シーケンスの長さで割る
def calc_score_normalized(word,sub_logits,get_score):
    return calc_score(word,sub_logits,get_score) / len(tokenizer.tokenize(word))

[
[
    calc_score_normalized("クェゼリン",logits[0,18:37,:],get_score_simple),
    calc_score_normalized("ケェヴェリン",logits[0,18:37,:],get_score_simple),
    calc_score_normalized("テスト",logits[0,18:37,:],get_score_simple)
],
[
    calc_score_normalized("クェゼリン",logits[0,18:37,:],get_score_by_rank),
    calc_score_normalized("ケェヴェリン",logits[0,18:37,:],get_score_by_rank),
    calc_score_normalized("テスト",logits[0,18:37,:],get_score_by_rank)
],
[
    calc_score_normalized("クェゼリン",logits[0,18:37,:],get_score_by_prob),
    calc_score_normalized("ケェヴェリン",logits[0,18:37,:],get_score_by_prob),
    calc_score_normalized("テスト",logits[0,18:37,:],get_score_by_prob)
]
]


まとめ

1. logitsのtopK(10)を取る（time_step*10のtensorになる）
2. それぞれのtime_stepのsoftmaxを取る
3. 検出キーワードの最初の文字から最後の文字までのprobを足し合わせるわけだが、どこからキーワードが始まっているかはまだ確定できない
4. 最初のトークンが始まるtime_stepを探す（＝topK内にトークンが現れる）あったら、そこのprobをスコアに足す
5. 次のtime_stepで次のトークンが存在したらprobを足して、次のトークンへ
6. 存在していなかった場合、前回の繰り返し or PADであれば、次のtimestepへ
7. どれも違う場合は、マッチ失敗とみなしスコアを放棄し、4へ
8. シーケンスの最後に到達した場合は、スコアをキーワードのスコアとして、appendする（複数検出するかもしれないので）
9. time_stepの最後に到達した場合は終了

基本的に、O(n) n = time_step 処理量になるので、キーワードが複数あるO(n^2)になる。まあまあ重いがtopKで刈り込んでるからいい？（複数キーワードは並列処理できるけど…）


In [95]:
def get_keyword_avgprobs(word,logits):
    ids=tokenizer.convert_tokens_to_ids(tokenizer.tokenize(word))
    logits_prob=torch.softmax(logits[0,:,:],dim=1)
    avg_probs=[]
    start_index=0
    id_index=0
    current_prob_total=0.0
    unmatch_count=0
    for i in range(len(logits[0])):
        pad_prob=logits_prob[i,tokenizer.pad_token_id]
        if id_index>0:
            last_id_prob=logits_prob[i,ids[id_index-1]]
        else:
            start_index=i
            last_id_prob=0.0
        current_prob=logits_prob[i,ids[id_index]]
        # 0.1以上の確率がある場合はマッチ、最初ではない場合は0.01でもよい
        if current_prob > 0.1 or (id_index>0 and current_prob > 0.01):
            #print(f"found {i} {tokenizer.convert_ids_to_tokens(ids[id_index])}({ids[id_index]}) top1 is {tokenizer.convert_ids_to_tokens(top1_id)}({top1_id}) current={current_prob} last={last_id_prob} pad={pad_prob} (total={current_prob_total})")
            current_prob_total+=current_prob
            #次へ
            id_index+=1
            if id_index==len(ids):
                #終わったので平均を計算して格納
                avg_probs.append({
                    "start": start_index,
                    "end": i,
                    "prob": (current_prob_total / len(ids)).item()
                })
                id_index=0
                current_prob_total=0.0
                unmatch_count=0                
        elif last_id_prob > 0.1 or pad_prob > 0.1:
            # PADや前回のIDの確率が0.1以上の場合は、何もしない
            pass
        else:
            #それ以外の場合は、マッチが失敗したとみなす
            #print(f"unmatch {i} expect {tokenizer.convert_ids_to_tokens(ids[id_index])}({ids[id_index]}) top1 is {tokenizer.convert_ids_to_tokens(top1_id)}({top1_id}) current={current_prob} last={last_id_prob} pad={pad_prob} ")
            if unmatch_count > 0:
                #print(f"unmatch count is {unmatch_count} so reset")
                id_index=0
                current_prob_total=0.0
                unmatch_count=0
            else:
                unmatch_count+=1
    return avg_probs

logits = model(input_values)[0]
get_keyword_avgprobs("アメリカ",logits)

[{'start': 209, 'end': 224, 'prob': 0.96728515625}]

In [88]:
# benchmark
import time
logits = model(input_values)[0]
start_time = time.time()
for i in range(100):
    get_keyword_avgprobs("クェゼリン",logits)
elapsed_time = time.time() - start_time
print(f"elapsed_time:{elapsed_time}[sec] avg_time:{elapsed_time/100}[sec/keyword]")


elapsed_time:11.659764051437378[sec] avg_time:0.11659764051437378[sec/keyword]
